In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scirpy as ir
from matplotlib import pyplot as plt, cm as mpl_cm
from cycler import cycler

sc.set_figure_params(figsize=(4, 4))
sc.settings.verbosity = 2  # verbosity: errors (0), warnings (1), info (2), hints (3)

In [36]:
from tcr_processing import *

In [40]:
def drop_dead(adata, count_threshold=False,mt_threshold=False):
    if not count_threshold:
        threshold=2500
    if not mt_threshold:
        mt_threshold=5
    adata.var['mt'] = adata.var_names.str.startswith('MT-')  # Generates a boolean mask for genes starting with 'MT'
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)  # Returns total_counts_['mt'] for vars fed in via qc_vars
    adata = adata[adata.obs.n_genes_by_counts < threshold, :]
    adata = adata[adata.obs.pct_counts_mt < mt_threshold, :]
    return adata

def preprocess(adata,min_cells=False,min_genes=False,drop_mt=False):

    if not min_cells:
        min_cells=10
    if not min_genes:
        min_genes=100


    sc.pp.filter_genes(adata, min_cells=min_cells)
    sc.pp.filter_cells(adata, min_genes=min_genes)

    if drop_mt:
        print('Dropping dead and doublet cells')
        adata=drop_dead(adata)

    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1000)
    sc.pp.log1p(adata)
    
    ir.tl.chain_qc(adata)
    print(
        "Fraction of cells with more than one pair of TCRs: {:.2f}".format(
            np.sum(
                adata.obs["chain_pairing"].isin(
                    ["extra VJ", "extra VDJ", "two full chains"]
                )
            )
            / adata.n_obs
        )
    )
    
    print('Dropping multichain and single chain receptors')

    # Drop multichain instances
    adata = adata[adata.obs["chain_pairing"] != "multichain", :].copy()

    # Drop single chain instances
    adata = adata[~adata.obs["chain_pairing"].isin(["orphan VDJ", "orphan VJ"]), :].copy()

    return adata

def load_data(contigs,feature_matrix):

    # Load the TCR data
    adata_tcr = ir.io.read_10x_vdj(contigs)

    # Load the associated transcriptomics data
    adata = sc.read_10x_h5(feature_matrix)

    # Merge
    ir.pp.merge_with_ir(adata, adata_tcr)

    print("Data loaded to Ann Object with shape: ",adata.shape)

    return adata

contigs="data/vdj_v1_hs_aggregated_donor2_all_contig_annotations.csv"
bc_matrix="data/vdj_v1_hs_aggregated_donor2_filtered_feature_bc_matrix.h5"

adata=load_data(contigs,bc_matrix)
adata=preprocess(adata,drop_mt=True)

reading data/vdj_v1_hs_aggregated_donor2_filtered_feature_bc_matrix.h5


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


 (0:00:05)


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/Users/danhudson/opt/miniconda3/envs/10X/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical
/Users/danhudson/opt/miniconda3/envs/10X/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'genome' as categorical


Data loaded to Ann Object with shape:  (91921, 33538)
filtered out 15493 genes that are detected in less than 10 cells


...

normalizing by total count per cell


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


    finished (0:00:01): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
Fraction of cells with more than one pair of TCRs: 0.07
Dropping multichain and single chain receptors


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Dropping dead and doublet cells


In [47]:
# Create barcode: epitope dictionary

def get_epitopes(binarized_matrix, savefile=False):
    '''Extract epitope map from binarized matrix file'''
    assert type(binarized_matrix)==str

    print('Loading binary matrix')
    binary_matrix=pd.read_csv(binarized_matrix)
    sub=binary_matrix[binary_matrix.columns[68:]]
    sub.set_index(binary_matrix['barcode'].values,inplace=True)
    print('Reading epitope specifity from %s files'%(len(sub)))
    eps=[sub.iloc[i][sub.iloc[i]==True] for i in range(len(sub))]
    assert len(eps)==len(sub)
    epitope={}
    for i in range(len(eps)):
        try:
            epitopes[eps[i].name]=eps[i].index[0].strip('_binder')
        except IndexError:
            epitopes[eps[i].name]='None'


    if savefile:
        save_pickle(epitopes,savefile)
    
    return binary_matrix, epitopes

binarized_matrix='data/vdj_v1_hs_aggregated_donor2_binarized_matrix.csv'
savefile='data/epitopes.pkl'

# bm,epitopes=get_epitopes(binarized_matrix,savefile)


In [48]:
# Load barcode: epitope dictionary
def load_pickle(loadfile):
    assert type(loadfile)==str
    print('Loading files from', loadfile)
    with open(loadfile,'rb') as f:
        data = pickle.load(f)
    return data

epitopes=load_pickle(savefile)

Loading files from data/epitopes.pkl


In [68]:
def get_TCRs(adata, savefile=False,epitope_dict = False):
    '''Extract TCRs from Ann object, and map to epitope specificity where provided'''
    TCR = adata.obs[['IR_VJ_1_j_call','IR_VJ_1_junction_aa','IR_VDJ_1_j_call','IR_VDJ_1_junction_aa']].copy()
    TCR.columns=['v.alpha','cdr3.alpha','v.beta','cdr3.beta']
    if epitope_dict:
        eps=[]
        for i in range(len(TCR)):
                if TCR.iloc[i].name in epitope_dict.keys():
                    eps.append(epitope_dict[TCR.index[i]])
                else:
                    eps.append('None')

        # epitopes = [epitope_dict[name] for name in CDR3.index if name in epitope_dict.keys() else None]
        TCR['Epitope']=eps
        adata.uns['TCRs']=TCR
    if savefile:
        print('Saving')
        TCR.to_pickle(savefile)
    return adata

adata = get_TCRs(adata,savefile='data/TCRs2.pkl',epitope_dict=epitopes)

Saving


In [52]:
# Compute scirpy clonotypes based on nucleotide sequence identity

ir.pp.ir_dist(adata)
ir.tl.define_clonotypes(adata, receptor_arms="all", dual_ir="primary_only")
ir.tl.clonotype_network(adata, min_cells=2)

Computing sequence x sequence distance matrix for VJ sequences.
Computing sequence x sequence distance matrix for VDJ sequences.
Initializing lookup tables. 
Computing clonotype x clonotype distances.
NB: Computation happens in chunks. The progressbar only advances when a chunk has finished. 


100%|██████████| 11205/11205 [00:18<00:00, 605.79it/s]


Stored clonal assignments in `adata.obs["clone_id"]`.


In [53]:
# Compute scirpy clonotypes based on amino acid sequence identity
ir.pp.ir_dist(
    adata,
    metric="identity",
    sequence="aa",
)

Computing sequence x sequence distance matrix for VJ sequences.
Computing sequence x sequence distance matrix for VDJ sequences.


In [9]:
adata.write_h5ad('data/anndata_object.h5ad')

/Users/danhudson/opt/miniconda3/envs/10X/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'receptor_type' as categorical
/Users/danhudson/opt/miniconda3/envs/10X/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'receptor_subtype' as categorical
/Users/danhudson/opt/miniconda3/envs/10X/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is 